In [2]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd

In [3]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[300:] 

In [4]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoCropper(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    '''
    x = sorted(data['x'], reverse = True)
    y = sorted(data['y'], reverse = True)
    
    # Try to exclude outliers
    for i in range(10):
        xMax = round(x[i + 1]) if x[i] - x[i + 1] >= 1 else round(x[i])
        
        xMin = round(x[-i-1]) if x[-i] - x[-i-1] >= 1 else round(x[-i])
            
        yMax = round(y[i + 1]) if y[i] - y[i + 1] >= 1 else round(y[i])

        yMin = round(y[-i-1]) if y[-i] - y[-i-1] >= 1 else round(y[-i])
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
    
    return xMin, yMin, lengthX, lengthY



In [13]:

testY = move_data['y'] - 54
testY[testY>0]

300      97.637673
301      96.464238
302      95.667854
303      92.945142
304      89.501368
305      87.452137
306      90.249456
307      91.781311
308      93.582320
309      98.509132
310     102.096745
311     107.472021
312     110.679431
313     115.733047
314     123.399451
315     125.071446
316     130.238227
317     133.969042
318     139.823178
319     142.310530
320     148.223490
321     156.740241
322     161.529191
323     165.602060
324     172.181792
325     175.397152
326     182.791460
327     192.989972
328     196.356635
329     197.946256
           ...    
3438    237.937450
3439    225.403668
3440    215.487085
3441    203.309465
3442    190.386603
3443    178.805893
3444    165.567668
3445    150.839678
3446    140.763852
3447    125.523216
3448    111.612663
3449     97.121559
3450     82.494046
3451     67.584011
3452     62.495194
3453     47.778252
3454     35.496385
3455     29.659161
3456     22.245843
3457     15.905497
3458      7.845531
3463      0.

In [12]:
test = move_data['x'] - 38
test[test>0]

300     102.824652
301     106.791032
302     108.281269
303     110.633805
304     116.011484
305     123.652093
306     129.687625
307     131.899760
308     135.042958
309     138.758209
310     140.622795
311     147.477197
312     148.505436
313     149.034113
314     154.621338
315     158.975889
316     163.151378
317     164.513144
318     171.282163
319     173.223418
320     178.214715
321     182.787089
322     188.195131
323     192.871386
324     201.488167
325     206.696326
326     208.687446
327     215.041439
328     220.996955
329     224.792359
           ...    
3451    112.839350
3452    117.237265
3453    116.633912
3454    113.695936
3455    114.969257
3456    111.707090
3457    110.629270
3458    106.231025
3459    107.423819
3460    110.270969
3461    111.707768
3462    117.584792
3463    121.665683
3464    132.005055
3465    139.129623
3466    142.512795
3467    150.832328
3468    152.728322
3469    164.136685
3470    169.844775
3471    175.558818
3472    181.

In [5]:
x, y, lengthX, lengthY = getCoord(move_data)

In [6]:
x, y, lengthX, lengthY

(54, 38, 331, 320)

In [83]:
convertX = 440/lengthX
convertY = 440/lengthY
print(convertX, convertY)

1.3293051359516617 1.375


In [80]:
440/331

1.3293051359516617

In [84]:
videoCropper('1034 SI_B, Aug 15, 13 7 49 rotated.mp4', "cropTest rotated.mp4", y, x, lengthY, lengthX)

1034 SI_B, Aug 15, 13 7 49 rotated.mp4 successfully  edited!


In [ ]:
##Question: choose length of the edge in order to be converted in video processing